In [1]:
import os
import time
os.chdir('models/efficientdet/')
import numpy as np
import torch
from torchvision import transforms
from src.dataset import WaymoDataset, Resizer, Normalizer
import gc
from torch.utils.data import DataLoader
dataset = WaymoDataset(cameras=['FRONT'], scope='validation', 
                transform=transforms.Compose([Normalizer(), Resizer(common_size=1920)])) #resize to 1920 1280
model = torch.load('trained_models/efficientdet_waymo.pth',
          map_location=torch.device('cpu'))
model.cuda()
model.requires_grad=False

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'src.model.EfficientDet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'src.model.EfficientNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [ ]:
import torch
from waymo_open_dataset.protos import metrics_pb2
from waymo_open_dataset import label_pb2,dataset_pb2
import concurrent.futures as concurr

# ['vehicle', 'pedestrian', 'sign', 'cyclist']
score_threshold = 0.3
def get_preds(images, model):
    start = time.time()
    with torch.no_grad():
        scores, annot, boxes = model(images.permute(0, 3, 1, 2).float())
    return scores, annot, boxes
def convert_to_waymo_boxes(boxes):
    tb = boxes.T
    length = tb[2]-tb[0]
    width = tb[3]-tb[1]
    tb[0] = tb[2]-0.5*length # center x
    tb[1] = tb[3]-0.5*width # center y
    tb[2] = length
    tb[3] = width
    res = tb.T
    return res

def gt_map(o, data):
    box = label_pb2.Label.Box()
    box.center_x = data.box.center_x
    box.center_y = data.box.center_y
    box.length = data.box.length
    box.width = data.box.width

    o.object.box.CopyFrom(box)
    o.object.type = data.type

def pred_map(o, data):
    s, c, b = data
    box = label_pb2.Label.Box()
    box.center_x = b[0]
    box.center_y = b[1]
    box.length = b[2]
    box.width = b[3]
    o.object.box.CopyFrom(box)
    o.score = s
    # Use correct type.
    o.object.type = c.item()+1

def filter_preds(data):
    scores, classes, boxes = data
    valid_idxs = []
    for c,cls in enumerate(classes):
        if (scores[c]>score_threshold):
            valid_idxs.append(c)
    valid_idxs = torch.tensor(valid_idxs).long()
    scores = scores.index_select(0, valid_idxs)
    classes = classes.index_select(0, valid_idxs)
    boxes = boxes.index_select(0, valid_idxs)
    return (scores, classes, boxes)

def process_frame_objects(objs, data, name, ts, cam, scope):
    if scope=='preds':
        data = filter_preds(data)
        num_objects = len(data[0])
        boxes = convert_to_waymo_boxes(data[2])
    else:
        num_objects = len(data)
    for c in range(num_objects):
        o = metrics_pb2.Object()
        o.context_name = name
        o.frame_timestamp_micros = ts
        o.camera_name = cam = cam
        if scope=='preds':
            data_i = (data[0][c], data[1][c], boxes[c]) # scores, classes, boxes
            pred_map(o, data_i)
        else:
            gt_map(o, data[c])
        objs.append(o)


def batch_frame_process(idx, objs):       
    print(idx)
    image_size = dataset[0]['img'].shape
    images = torch.zeros((idx[1]-idx[0],image_size[0], image_size[1], image_size[2]),
                         dtype=torch.float64, device=torch.device('cuda'), requires_grad=False)
    items = []
    print(images.shape)
    for index in list(range(idx[0], idx[1])):
        item = dataset[index]
        items.append(item)
        images[index][:] = item['img']
    images = torch.DoubleTensor(images)
    preds = get_preds(images, model)
    print(preds.shape)
    
    for ind, item in enumerate(items):
        ctx = item['raw']['context']
        cam = getattr(dataset_pb2.CameraName, dataset.get_cam_type(idx))
        # Populate preds
        preds[ind][2] /= item['scale']
        process_frame_objects(objs, preds[ind], ctx['name'],
                              ctx['timestamp'], cam, 'preds')
    
    
def single_batch_process(idx_range):
    try:
        batch_idx = idx_range[2]
        pred_objects = metrics_pb2.Objects()
        batch_frame_process((idx_range[0], idx_range[1]), pred_objects.objects)

        # Write objects to a file.
        print(pred_objects)
#         f = open(f'/home/project_x/evaluation/results/preds{shard_idx}.bin', 'wb')
#         f.write(pred_objects.SerializeToString())
#         f.close()
#         gc.collect()
    except EOFError:
        print(f"EOFERROR on item {c} from shard {shard_idx}")
    return shard_idx


num_frames = len(dataset)
batch_size = 1
batches = int(np.ceil(num_frames/batch_size))
batch_indices = [[i*batch_size, (i+1)*batch_size, i] for i in range(0,batches)]
batch_indices[-1][-2] = num_frames-1

single_batch_process(batch_indices[0])

# concur= True
# start = time.time()
# with concurr.ProcessPoolExecutor(max_workers = 2) as executor:
#     results = executor.map(single_batch_process, batch_indices)
#     for i in results:
#         print(f'Compiled and save shard {i}')
# print("Done compiling results")
# end = time.time()
# print(f"Time taken: {end-start}")


(0, 1)
torch.Size([1, 1920, 1920, 3])


In [ ]:
import os
import psutil
process = psutil.Process(os.getpid())
print(process.memory_info().rss) 

In [ ]:
7325458432
425205760
443990016
443338752
841973760

In [ ]:
3126.1820917129517/60

In [ ]:
%%timeit
get_preds(dataset[1], model)

In [ ]:
0.4 *10000

In [ ]:
3126.1820917129517/60

In [ ]:
num_frames = len(dataset)
div = 1000
shards = int(np.ceil(num_frames/div))
shard_indices = [[i*div, (i+1)*div, i] for i in range(10,shards)]
shard_indices[-1][-2] = num_frames-1

In [ ]:
shard_indices

In [ ]:
for i in range(16995, 17005):
    print(dataset[i].keys())

In [ ]:
dataset[1]['id']

In [ ]:
type(EOFError)

In [ ]:
9035/60

In [ ]:
x = """/home/project_x/evaluation/results/preds0.bin /home/project_x/evaluation/results/preds1.bin /home/project_x/evaluation/results/preds10.bin /home/project_x/evaluation/results/preds11.bin /home/project_x/evaluation/results/preds12.bin /home/project_x/evaluation/results/preds13.bin /home/project_x/evaluation/results/preds14.bin /home/project_x/evaluation/results/preds15.bin /home/project_x/evaluation/results/preds16.bin /home/project_x/evaluation/results/preds17.bin /home/project_x/evaluation/results/preds18.bin /home/project_x/evaluation/results/preds19.bin /home/project_x/evaluation/results/preds2.bin /home/project_x/evaluation/results/preds20.bin /home/project_x/evaluation/results/preds21.bin /home/project_x/evaluation/results/preds22.bin /home/project_x/evaluation/results/preds23.bin /home/project_x/evaluation/results/preds24.bin /home/project_x/evaluation/results/preds25.bin /home/project_x/evaluation/results/preds26.bin /home/project_x/evaluation/results/preds27.bin /home/project_x/evaluation/results/preds28.bin /home/project_x/evaluation/results/preds29.bin /home/project_x/evaluation/results/preds3.bin /home/project_x/evaluation/results/preds30.bin /home/project_x/evaluation/results/preds31.bin /home/project_x/evaluation/results/preds32.bin /home/project_x/evaluation/results/preds33.bin /home/project_x/evaluation/results/preds34.bin /home/project_x/evaluation/results/preds35.bin /home/project_x/evaluation/results/preds36.bin /home/project_x/evaluation/results/preds37.bin /home/project_x/evaluation/results/preds38.bin /home/project_x/evaluation/results/preds39.bin /home/project_x/evaluation/results/preds4.bin /home/project_x/evaluation/results/preds5.bin /home/project_x/evaluation/results/preds6.bin /home/project_x/evaluation/results/preds7.bin /home/project_x/evaluation/results/preds8.bin /home/project_x/evaluation/results/preds9.bin"""
x.replace(' ', '","')

In [ ]:
# gcloud compute scp t.sautory@gpu-2:/../home/project_x/README.md Desktop